In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName('listing migration') \
  .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 14:45:12 INFO SparkEnv: Registering MapOutputTracker
24/01/21 14:45:12 INFO SparkEnv: Registering BlockManagerMaster
24/01/21 14:45:12 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/01/21 14:45:12 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
rawdata_bucket = 'airbnb-raw-data'   
prefix = 'japan/27-12-2023/'    # Should be parameterized

In [8]:
from google.cloud import storage

storage_client = storage.Client()
blobs = storage_client.list_blobs(rawdata_bucket,prefix=prefix)

for blob in blobs:
    if not blob.name.endswith('/'):
        print(blob.name)

japan/27-12-2023/calendar.csv
japan/27-12-2023/listings.csv
japan/27-12-2023/neighbourhoods.geojson
japan/27-12-2023/reviews.csv


In [28]:
import pandas as pd
csv_data = pd.read_csv(f"gs://{rawdata_bucket}/japan/27-12-2023/listings.csv")
org_fields = """id listing_url name description neighborhood_overview picture_url host_id neighbourhood_cleansed 
latitude longitude property_type room_type amenities""".split()
print(org_fields,len(org_fields))
pd_df = csv_data.filter(items=org_fields, axis=1) 

['id', 'listing_url', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'amenities'] 13


In [30]:
pd_df

,id,listing_url,name,description,neighborhood_overview,picture_url,host_id,neighbourhood_cleansed,latitude,longitude,property_type,room_type,amenities
0,197677,https://www.airbnb.com/rooms/197677,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,Sumida Ku,35.717070,139.826080,Entire rental unit,Entire home/apt,[]
1,776070,https://www.airbnb.com/rooms/776070,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,NaN,We love Nishinippori because is nearer to Toky...,https://a0.muscache.com/pictures/efd9f039-dbd2...,801494,Kita Ku,35.738440,139.769170,Private room in home,Private room,[]
2,905944,https://www.airbnb.com/rooms/905944,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,Shibuya Ku,35.678780,139.678470,Entire rental unit,Entire home/apt,[]
3,1016831,https://www.airbnb.com/rooms/1016831,Home in Setagaya · ★4.95 · 1 bedroom · 2 beds ...,NaN,The location is walkable distance to famous Sh...,https://a0.muscache.com/pictures/airflow/Hosti...,5596383,Setagaya Ku,35.658000,139.671340,Private room in home,Private room,[]
4,3838856,https://www.airbnb.com/rooms/3838856,Home in Shinjuku · ★4.89 · 3 bedrooms · 5 beds...,NaN,The house is in a quiet and safe street. The s...,https://a0.muscache.com/pictures/miso/Hosting-...,4626879,Shibuya Ku,35.682450,139.690090,Entire home,Entire home/apt,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13720,1052613009241026699,https://www.airbnb.com/rooms/1052613009241026699,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,★概要★<br />サブカルチャーで有名な秋葉原周辺には大変魅力的な場所がたくさんあります。...,https://a0.muscache.com/pictures/miso/Hosting-...,118634275,Shinjuku Ku,35.693670,139.711850,Entire rental unit,Entire home/apt,[]
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,Taito Ku,35.719754,139.789284,Entire rental unit,Entire home/apt,[]
13722,1052621352644395645,https://www.airbnb.com/rooms/1052621352644395645,Rental unit in Shinjuku City · ★New · 2 bedroo...,NaN,新宿駅直通のバス停が徒歩2分の距離にあり、とてもアクセスしやすいです。バスストップから新宿駅...,https://a0.muscache.com/pictures/c6c689cd-2cd4...,320838422,Nakano Ku,35.710190,139.692800,Entire rental unit,Entire home/apt,[]
13723,1052624489140939071,https://www.airbnb.com/rooms/1052624489140939071,Rental unit in Shinjuku City · ★New · 3 bedroo...,NaN,【新宿へようこそ！】<br />新宿は20世紀に大幅に開発され、今や東京の代名詞的な存在とな...,https://a0.muscache.com/pictures/miso/Hosting-...,320838422,Shinjuku Ku,35.698234,139.700994,Entire rental unit,Entire home/apt,[]


In [33]:
table = 'dim_calendar'
dataset = 'airbnb'
gcs_staging_bucket = 'airbnb-staging-data'
gcs_staging_data_path = 'gs://{}/{}listing.parquet'.format(gcs_staging_bucket,prefix)
print(gcs_staging_data_path)
pd_df.to_parquet(gcs_staging_data_path)

gs://airbnb-staging-data/japan/27-12-2023/listing.parquet


24/01/21 03:56:27 WARN BigQueryDataSourceWriterInsertableRelation: It seems that 2 out of 0 partitions have failed, aborting
24/01/21 03:56:27 WARN BigQueryDirectDataSourceWriterContext: BigQuery Data Source writer dbd14af1-cada-4854-b6c9-ff921f090c16 aborted
